In [ ]:
# http://www.dabeaz.com/coroutines/
# A Curious Course on Coroutines and Concurrency

In [2]:
# A Python version of Unix 'tail -f'
# follow.py file 
import time 

def follow(thefile):
    # f.seek(offset, from_what)
    #   offset: The position is computed from adding offset to a reference point
    #   from_what: 0 measures from begining of the file, 1 measures from current file position, 2 measure  end of the file as the reference point.
    thefile.seek(0,2) # go to the end of the file 
    while True:
        line = thefile.readline()
        if not line:
            time.sleep(0.1) # Sleep briefly
            continue
        yield line 

logfile = open("data-set/access-log.txt")
for line in follow(logfile):
    print(line)



access-log 08


KeyboardInterrupt: 

# Generators as Pipelines

In [3]:
# Print all server log entries containing 'log'
# pipeline.py 

def grep(pattern, lines):
    print("Looking for %s" % pattern)
    for line in lines:
        if pattern in line:
            yield line

# Set up a processing pipe : tail -f | grep log 
logfile = open("data-set/access-log.txt")
loglines = follow(logfile)
pylines = grep("log", loglines)

# Pull results out of the processing pipeline 
for line in pylines:
    print(line)


access-log 09
access-log 10
access-log 11


KeyboardInterrupt: 

In [5]:
# Yield as an Expression 

def grep(pattern):
    print("Looking for %s" %pattern)
    while True:
        line = (yield) # use yield as an expression 
        if pattern in line:
            print(line)
g = grep("python")
g.__next__()
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")



Looking for python
python generators rock!


In [11]:
# Coroutine Execution
#   They only run in response to next() and send() methods 
# Solved by wrapping coroutines with a decorator 

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args,**kwargs)
        cr.__next__()
        return cr 
    return start 

@coroutine
def grep(pattern):
    print("Looking for %s" %pattern)
    try:
        while True:
            line = (yield) # use yield as an expression 
            if pattern in line:
                print(line)
    except GeneratorExit: # Catching close() 
        print("Going away. Goodbye")

# A coroutine might run indefinitely 
g = grep("python")
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")
# Closing a Coroutine 
g.close()

Looking for python
python generators rock!
Going away. Goodbye


In [15]:
# Throwing an Exception 
# Coroutine Execution
#   They only run in response to next() and send() methods 
# Solved by wrapping coroutines with a decorator 

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args,**kwargs)
        cr.__next__()
        return cr 
    return start 

@coroutine
def grep(pattern):
    print("Looking for %s" %pattern)
    try:
        while True:
            line = (yield) # use yield as an expression 
            if pattern in line:
                print(line)
    except GeneratorExit: # Catching close() 
        print("Going away. Goodbye")

# A coroutine might run indefinitely 
g = grep("python")
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")
# Throwing an Exception 
g.throw(RuntimeError,"You're hosed") 

Looking for python
Going away. Goodbye
python generators rock!


RuntimeError: You're hosed

In [16]:
# nterlude
# Generators produce values 
# Coroutines tend to consume values 

def countdown(n):
    print("Counting down from", n)
    while n >= 0:
        newvalue = (yield n)
        # If a new value got sent in, reset n with it 
        if newvalue is not None:
            n = newvalue 
        else:
            n -= 1 
            
c = countdown(5)
for n in c:
    print(n)
    if n == 5:
        c.send(3)


Counting down from 5
5
2
1
0


In [22]:
# Part 2 
# Coroutines, Pipelines, and Dataflow 
# A source that mimics Unix 'tail -f'

import time 

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args,**kwargs)
        cr.__next__()
        return cr 
    return start 

def follow(thefile, target):
    thefile.seek(0, 2) # Go to the end of the file 
    while True:
        line = thefile.readline() 
        if not line:
            time.sleep(0.1) # Sleep briefly 
            continue
        target.send(line)

# A sink that just prints the lines
@coroutine
def printer():
    while True:
        line = (yield) # Receive an item 
        print(line)

@coroutine
def filter(target):
    while True:
        item = (yield) # Receive an item
        if item in "fork idiot":
            pass # Transform/filter item
        else:
            # Send it along to the next stage 
            target.send(item)

@coroutine
def grep(pattern, target):
    print("Looking for %s" %pattern)
    try:
        while True:
            line = (yield) # use yield as an expression 
            if pattern in line:
                target.send(line) # Send to next stage 
    except GeneratorExit: # Catching close() 
        print("Going away. Goodbye")
        
f = open("data-set/access-log.txt")
#follow(f, printer())
follow(f, grep('python', printer()))
# Critical point:
# follow() is driving the entire computation by reading lines and pushing them into the printer() coroutine 
        

Looking for python
python-alala


KeyboardInterrupt: 

In [23]:
# Interlude 
## Coroutines flip generators around 
## Key difference. Generators pull data through the pipe with iteration. Coroutines push data into the pipeline with send()

# Broadcasting 
@coroutine
def broadcast(targets):
    while True:
        item = (yield)
        for target in targets: # This takes a sequence of coroutines (targets) and sends received items to all of them
            target.send(item)
f = open("data-set/access-log.txt")
follow(f, broadcast([
    grep('python', printer()),
    grep('ply', printer()),
    grep('swig', printer())
]))

Looking for python
Looking for ply
Looking for swig
ply ...1
python ...2
swig...3


KeyboardInterrupt: 

In [29]:
# A Digression 
import timeit

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args,**kwargs)
        cr.__next__()
        return cr 
    return start 

# A sink that just prints the lines
@coroutine
def printer():
    while True:
        line = (yield) # Receive an item 
        print(line)

# handler object 
class GrepHandler(object):
    def __init__(self, pattern, target):
        self.pattern = pattern
        self.target = target 
    
    def send(self, line):
        if self.pattern in line:
            self.target.send(line)

# The coroutine version
# faster 
@coroutine 
def grep(pattern, target):
    while True:
        line = (yield)
        if pattern in line:
            target.send(line)

@coroutine
def null():
    while True: item = (yield)

line = 'Python is nice'
p1 = grep('python', null()) # coroutine 
p2 = GrepHandler('python', null()) # Object 

# Send in 1,000,000 lines
%timeit "p1.send(line)", "from __main__ import line, p1"
%timeit "p2.send(line)", "from __main__ import line, p2"

20.2 ns ± 0.444 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
20.5 ns ± 1.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [30]:
# Part 3
## Coroutines and Event Dispatching 
## Coroutines can be used to write various components that process event streams

# SAX -- There are many ways to parse XML
# An old-school approach: SAX
# SAX is an event driven interface 

# Minimal SAX Example 
import xml.sax

class MyHandler(xml.sax.ContentHandler):
    
    def startElement(self, name, attrs):
        print("startElement", name)
    
    def endElement(self, name):
        print("endElement", name)
    
    def characters(self, text):
        print("characters", repr(text)[:40])

xml.sax.parse("data-set/somefile.xml", MyHandler())


Going away. Goodbye
Going away. Goodbye
Going away. Goodbye
Going away. Goodbye
startElement buses
characters '\n'
characters '\t\t'
startElement bus
characters '\n'
characters '\t\t\t'
startElement id
characters '7574'
endElement id
characters '\n'
characters '\t\t\t'
startElement route
characters '147'
endElement route
characters '\n'
characters '\t\t\t'
startElement color
characters '#3300ff'
endElement color
characters '\n'
characters '\t\t\t'
startElement revenue
characters 'true'
endElement revenue
characters '\n'
characters '\t\t\t'
startElement direction
characters 'North Bound'
endElement direction
characters '\n'
characters '\t\t\t'
startElement latitude
characters '41.925682067871094'
endElement latitude
characters '\n'
characters '\t\t\t'
startElement longitude
characters '-87.63092803955078'
endElement longitude
characters '\n'
characters '\t\t\t'
startElement pattern
characters '2499'
endElement pattern
characters '\n'
characters '\t\t\t'
startElement patternDirection
cha

In [81]:
# From SAX to Coroutines 
## Can dispatch SAX events into coroutines 
import timeit 

import xml.sax
# XML Parsing with Expat 
# expat is low-level (a C extension module)
import xml.parsers.expat  

def expat_parse(f, target):
    parser = xml.parsers.expat.ParserCreate()
    parser.buffer_size = 65536
    parser.buffer_text = True 
    parser.StartElementHandler = lambda name,attrs: target.send(('start', (name, attrs)))
    parser.EndElementHandler = lambda name: target.send(('end', name))
    parser.CharacterDataHandler = lambda data: target.send(('text', data))
    # The expar parser in xml.parsers.expat has a Parse method and a ParseFile method. The Parse method parses a string, however the ParseFile method wants bytes.
    parser.ParseFile(f)

class EventHandler(xml.sax.ContentHandler):
    
    def __init__(self, target):
        self.target = target 
    
    def startElement(self, name, attrs):
        self.target.send(('start', (name, attrs._attrs)))
    
    def characters(self, text):
        self.target.send(('text', text))
    
    def endElement(self, name):
        self.target.send(('end', name))
    
def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args,**kwargs)
        cr.__next__()
        return cr 
    return start 
        
@coroutine
def buses_to_dicts(target):
    while True:
        event, value = (yield)
        # Look for the start of a <bus> element 
        if event == 'start' and value[0] == 'bus':
            busdict = {}
            fragments = []
            # Capture text of inner elements in a dict 
            while True:
                event, value = (yield)
                if event == 'start': 
                    fragments = []
                elif event == 'text':
                    fragments.append(value)
                elif event == 'end':
                    if value != 'bus':
                        busdict[value] = "".join(fragments)
                    else:
                        target.send(busdict)
                        break 

# A sink that just prints the lines
@coroutine
def printer():
    while True:
        line = (yield) # Receive an item 
        print(line)

# Filtering Elements 
@coroutine
def filter_on_field(fieldname,value,target):
    while True:
        d = (yield)
        if d.get(fieldname) == value:
            target.send(d)

# Processing Elements 
@coroutine
def bus_locations():
    while True:
        bus = (yield)
        print("%(route)s,%(id)s,\"%(direction)s\",\"%(latitude)s,%(longitude)s" %bus)
        print('{route},{id},{direction},{latitude},{longitude}'.format(**bus))
            
# SAX version 
%time xml.sax.parse("data-set/somefile.xml",EventHandler(buses_to_dicts(filter_on_field("route","147",filter_on_field("direction","North Bound",bus_locations())))))

# Expat version 
%time expat_parse(open("data-set/somefile.xml", 'rb'),buses_to_dicts(filter_on_field("route","147",filter_on_field("direction","North Bound",bus_locations()))))

147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.42 ms
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925

In [80]:
# Part 4 
# From Data Processing to Concurrent Programming 
# The Story So Far 
# Coroutines are similar to generators 
# You can create collections of small processing components and connect them together
# You can process data by setting up pipelines dataflow graphs 
# You send data to coroutines
# You send data to threads (via queues)
# You send data to processes (via message)

# Basic Concurrency 并发
# You can package coroutines inside threads or subprocesses by adding extra layers 

# The queue module implements multi-producer,multi-consumer queues
from queue import Queue 
from threading import Thread 

def coroutine(func): 
    def start(*args, **kwargs): 
        cr = func(*args,**kwargs) 
        cr.__next__() 
        return cr 
    return start 

@coroutine 
def threaded(target): 
    messages = Queue() # A message queue; Constructor for a FIFO queue    
    def run_target(): # A thread.Loop forever, pulling items out of the message queue and sending them to the target
        while True:
            item = messages.get()
            if item is GeneratorExit: # Handle close() so that the thread shutdown correctly 
                target.close()
                return
            else:
                target.send(item)
    Thread(target=run_target).start()
    try: # Receive items and pass them into the thread (via the queue)
        while True:
            item = (yield)
            messages.put(item)
    except GeneratorExit:
        messages.put(GeneratorExit)

# SAX Thread version 
%time xml.sax.parse("data-set/somefile.xml",EventHandler(buses_to_dicts(threaded(filter_on_field("route","147",filter_on_field("direction","North Bound",bus_locations()))))))


CPU times: user 2 ms, sys: 999 µs, total: 3 ms
Wall time: 2.51 ms
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078
147,7574,"North Bound","41.925682067871094,-87.63092803955078
147,7574,North Bound,41.925682067871094,-87.63092803955078
147,7575,"North Bound","41.925682067871094,-87.63092803955078
147,7575,North Bound,41.925682067871094,-87.63092803955078


In [82]:
# A subprocess Target 
# Bridge two coroutines over a file/pipe 
# Of course, the devil is in the details 细节决定成败

# The pickle module implements binary protocols for serializing and de-serializing a Python object structure
import pickle # Python object serialization 

def coroutine(func): 
    def start(*args, **kwargs): 
        cr = func(*args,**kwargs) 
        cr.__next__() 
        return cr 
    return start 

@coroutine
def sendto(f):
    try:
        while True:
            item = (yield)
            # pickle.dump(obj, file) Write a pickled representation of obj to the open file object file 
            pickle.dump(item, f)
            f.flush()
    except StopIteration:
        f.close()
        
def recvfrom(f, target):
    try:
        while True:
            # pickle.load(file,) Read a pickled object representation from the open file object file and return the reconstituted object hierarchy specified therein
            item = pickle.load(f)
            target.send(item)
    except EOFError:
        target.close()

        

In [91]:
# Some Hidden Dangers 隐藏的bug
# The send() method on a coroutine must be properly synchronized 
# If you can send() on an already-executing coroutine, your program will crash 

# Multiplpe threads sending data into the same target coroutine 
# cocrash.py
# An example of hooking coroutines up in a way that might cause a potential
# crash. Basically there are two threads feeding data into the printer() coroutine 

from queue import Queue 
from threading import Thread 

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        cr.__next__()
        return cr
    return start 

@coroutine 
def threaded(target): 
    messages = Queue() # A message queue; Constructor for a FIFO queue    
    def run_target(): # A thread.Loop forever, pulling items out of the message queue and sending them to the target
        while True:
            item = messages.get()
            if item is GeneratorExit: # Handle close() so that the thread shutdown correctly 
                target.close()
                return
            else:
                target.send(item)
    Thread(target=run_target).start()
    try: # Receive items and pass them into the thread (via the queue)
        while True:
            item = (yield)
            messages.put(item)
    except GeneratorExit:
        messages.put(GeneratorExit)

# A sink that just prints the lines
@coroutine
def printer():
    while True:
        line = (yield) # Receive an item 
        print(line)

# Broadcast a stream onto multiple targets
@coroutine
def broadcast(targets):
    while True:
        item = (yield)
        for target in targets:
            target.send(item)

# The coroutine version
# faster 
@coroutine 
def grep(pattern, target):
    while True:
        line = (yield)
        if pattern in line:
            target.send(line)
        
p = printer()
target = broadcast([threaded(grep('foo',p)),
                   threaded(grep('bar',p))])

# Ajust the count if this doesn't cause a crash 
for i in range(10):
    target.send("foo is nice\n")
    target.send("bar is bad\n")

del target
del p 

# Limitations
# You can't create loops or cycle 
# Stacked sends are building up a kind of call-stack (send() doesn't return until the target yields)
# If you call a coroutine that's already in the process of sending, you'll get an error
# send() doesn't suspend coroutine execution 


foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

foo is nice

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad

bar is bad



In [ ]:
# Part 5
# Coroutines as Tasks 
# Tasks have a few essential features
#     Independent control flow 
#     Internal state 
#     Can be scheduled (suspended/resumed)
#     Can communicate with other tasks 

def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        cr.__next__()
        return cr 
    return start

@coroutine
def grep(pattern): # A coroutine is just a sequence of statements like any other Python function 
    print("Looking for %s" %pattern)
    while True:
        line = (yield) # yield expressions receive input 
        if pattern in line: 
            print(line)
# Coroutine can be suspended and resumed 
# yield suspends execution
# send() resumes execution 
# close() terminates execution 

In [ ]:
#Part 6
# A Crash Course in Operating Systems 
# On a CPU, a Program is a series of instructions 
# When runing, there is no notion of doing more than one thing at a time (or any kind of task switching)

# The Multitasking problem 
# CPUs don't know anything about multitasking
# Nor do application programs 
# Well, surely something has to know about it
# Hint: It's the operating system 

# Operating Systems
# As you hopefully know, the operating system(e.g., Linux, Window) is responsible for running programs on your machine 
# And as you have observed, the operating system does allow more than one process to execute at once.
# It does this by rapidly switching between tasks 

# Question: How does it do that?
# A Conundrum 
# When a CPU is running your program, it is not running the operationg system 

# Question: How does the operating system(which is not running) make an application(which is running) switch to another task?
# The "context-switching" problem 

# Interrupts and Traps 
# There are usually only two mechanisms that an operating system uses to gain control
# Interrupts - Some kind of hardware related signal (data received, timer, keypress, etc)
# Traps - A software generated signal 
# In both cases, the CPU briefly suspends what it is doing, and runs code that's part of the OS
# It is at this time the OS might switch tasks 

# High Level Overview 

# Task Scheduling 
# To run many tasks, add a bunch of queues 
# If you treat yield as a trap, you can build a multitasking "operating system" --all in Python

In [97]:
# Part 7 
# Let's Build an Operating system 
# Our Challenge 
# Build a multitasking "operating system"
# Use nothing but pure Python code 
# No threads
# No subprocesses 
# Use generators/coroutine 

# Some Motivation 
# There has been a lot of recent interest in alternatives to threads (especially due to the GIL)
# Non-blocking and asynchronous I/O
# Example: servers capable of supporting thousands of simultaneous client connections 
# A lot of work has focused on event-driven systems or the "Reactor Model"(e.g., Twisted)

from queue import Queue

# A task object 
class Task(object):
    taskid = 0
    def __init__(self, target):
        Task.taskid += 1
        self.tid      = Task.taskid  # Task ID
        self.target   = target       # Target coroutine 
        self.sendval  = None         # Value to send 
    
    def run(self):
        return self.target.send(self.sendval)

# The Scheduler 
class Scheduler(object):
    def __init__(self):
        self.ready = Queue() # A queue of tasks that are ready to run 
        self.taskmap = {} # A dictinary that keeps track of all active tasks (each task has a unique integer task ID)
    
    def new(self, target): # Introduces a new task to the scheduler 
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask 
        self.schedule(newtask)
        return newtask.tid 
    
    def schedule(self, task):
        self.ready.put(task) # Put a task onto the ready queue, This makes it available to run 
    
    def exit(self, task): # Remove the task from the scheduler's task map 
        print("Task %d terminated" %task.tid)
        del self.taskmap[task.tid]
    
    def mainloop(self): # The main scheduler loop. It pulls tasks off the queue and runs them to the next yield 
        while self.taskmap:
            task = self.ready.get()
            try: # Catch task exit and cleanup
                result = task.run()
            except StopIteration:
                self.exit(task)
                continue
            self.schedule(task)
    
    
    
# Task Example 
# A very simple generator 
def foo():
    print("Part 1")
    yield
    print("Part 2")
    yield 

t1 = Task(foo()) # Wrap in a Task 
t1.run()
t1.run()

# First Multitasking 
# Two tasks
def task1():
#     while True:
#         print("I'm task1")
#         yield
    for i in range(10):
        print("I'm task1")
        yield

def task2():
    while True:
        print("I'm task2")
        yield 

# Running them into the shceduler
sched = Scheduler()
sched.new(task1())
sched.new(task2())
sched.mainloop()
# Example output 


Part 1
Part 2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
I'm task1
I'm task2
Task 2 terminated
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task

I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


...

I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2


I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2
I'm task2

KeyboardInterrupt: 

In [100]:
# Second Multitasking
def foo():
    for i in range(10):
        print("I'm foo")
        yield

def bar():
    for i in range(5):
        print("I'm bar")
        yield

sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

I'm foo
I'm bar
I'm foo
I'm bar
I'm foo
I'm bar
I'm foo
I'm bar
I'm foo
I'm bar
I'm foo
Task 7 terminated
I'm foo
I'm foo
I'm foo
I'm foo
Task 6 terminated


In [4]:
# System Calls
# In a real operating system, traps are how application programs request the services of the operating system (syscalls)
# In our code, the scheduler is the operating system and the yield statment is a trap
# To request the service of the scheduler, tasks will use the yield statement with a value 
from queue import Queue

# A task object 
class Task(object):
    taskid = 0
    def __init__(self, target):
        Task.taskid += 1
        self.tid      = Task.taskid  # Task ID
        self.target   = target       # Target coroutine 
        self.sendval  = None         # Value to send 
    
    def run(self):
        return self.target.send(self.sendval)

# The Scheduler 
class Scheduler(object):
    def __init__(self):
        self.ready = Queue() # A queue of tasks that are ready to run 
        self.taskmap = {} # A dictinary that keeps track of all active tasks (each task has a unique integer task ID)
        self.exit_waiting = {} # This is a holding area for tasks that are waiting. A dict mapping task ID to tasks waiting for exit.
        
    def new(self, target): # Introduces a new task to the scheduler 
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask 
        self.schedule(newtask)
        return newtask.tid 
    
    def schedule(self, task):
        self.ready.put(task) # Put a task onto the ready queue, This makes it available to run 
    
    def exit(self, task): # Remove the task from the scheduler's task map 
        print("Task %d terminated" %task.tid)
        del self.taskmap[task.tid]
        # Notify other tasks waiting for exit 
        # When a task exits, we pop a list of all waiting tasks off out of the waiting area and reschedule them
        for task in self.exit_waiting.pop(task.tid, []):
            self.schedule(task)
    
    def waitforexit(self, task, waittid): # A utility method that makes a task wait for another task. It puts the task in the waiting area 
        if waittid in self.taskmap:
            self.exit_waiting.setdefault(waittid,[]).append(task)
            return True 
        else:
            return False 
        
    def mainloop(self): # The main scheduler loop. It pulls tasks off the queue and runs them to the next yield 
        while self.taskmap:
            task = self.ready.get()
            try: # Catch task exit and cleanup
                result = task.run() # Look at the result yielded by the task. If it's a SystemCall, do some setup and run the system call on behalf of the task
                if isinstance(result, SystemCall):
                    result.task = task  # These attributes hold information about the environment (current task and scheduler)
                    result.sched = self
                    result.handle()
                    continue 
            except StopIteration:
                self.exit(task)
                continue
            self.schedule(task)

class SystemCall(object): # System Call base class. All system operations will be operations will be implemented by inheriting from this class 
    def handle(self):
        pass 

# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)

# A First System Call
def foo():
    mytid = yield GetTid()
    while True:
        print("I'm foo", mytid)
        yield 

def bar():
    mytid = yield GetTid()
    for i in range(10):
        print("I'm bar", mytid)
        yield 

# sched = Scheduler()
# sched.new(foo())
# sched.new(bar())
# sched.mainloop() # each task has a different task id 

# Design Discussion
# Real operating systems have a strong notion of "protection" (e.g., memory protection)
# Application programs are not strongly linked to the OS kernel (traps are only interfaced)
# For sanity, we are going to emulate this
#   Tasks do not see the scheduler 
#   Tasks do not see other tasks 
#   yield is the only external interface 

# Task Management
# Let's make more some system calls 
# Some task management functions
#   Create a new task 
#   Kill an existing task 
#   Wait for a task to exit 
# There mimic common operations with threads or processes 

# Create a another system call 
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target 
    
    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid 
        self.sched.schedule(self.task)

# Killing Tasks 
class KillTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid 
    
    def handle(self):
        task = self.sched.taskmap.get(self.tid, None)
        if task:
            task.target.close()
            self.task.sendval = True
        else:
            self.task.sendval = False 
        self.sched.schedule(self.task)

# Waiting for Tasks 
class WaitTask(SystemCall):
    def __init__(self, tid):
        self.tid = tid 
    
    def handle(self):
        result = self.sched.waitforexit(self.task, self.tid)
        self.task.sendval = result 
        # If waiting for a non-existent task,
        # return immediately without waiting 
        if not result:
            self.sched.schedule(self.task)
    

def bar():
    while True:
        print("I'm bar")
        yield 

def main():
    child = yield NewTask(foo()) # Launch new task 
    for i in range(5):
        yield 
    yield KillTask(child)        # Kill the task 
    print("Main done")

sched = Scheduler()
sched.new(main())
sched.mainloop()

I'm foo 2
I'm foo 2
I'm foo 2
I'm foo 2
I'm foo 2
Task 2 terminated
Main done
Task 1 terminated


In [7]:
# An Echo Server Attempt 
import socket 
from queue import Queue
# This module provides access to the select() and poll() functions 
import select # Waiting for I/O completion 

# A task object 
class Task(object):
    taskid = 0
    def __init__(self, target):
        Task.taskid += 1
        self.tid      = Task.taskid  # Task ID
        self.target   = target       # Target coroutine 
        self.sendval  = None         # Value to send 
    
    def run(self):
        return self.target.send(self.sendval)

# The Scheduler 
class Scheduler(object):
    def __init__(self):
        self.ready = Queue() # A queue of tasks that are ready to run 
        self.taskmap = {} # A dictinary that keeps track of all active tasks (each task has a unique integer task ID)
        self.read_waiting = {} # holding areas for tasks blocking on I/O. These are dictionaries mapping file descriptors to tasks
        self.write_waiting = {} 
        self.exit_waiting = {} # This is a holding area for tasks that are waiting. A dict mapping task ID to tasks waiting for exit.
    
    def waitforread(self, task, fd):
        self.read_waiting[fd] = task 
    
    def waitforwrite(self, task, fd):
        self.write_waiting[fd] = task 
    
    def iopoll(self, timeout): # I/O Polling. Use select() to determine which file descriptors can be used. Unblock any associated task
        if self.read_waiting or self.write_waiting:
            # Poll for I/O activity 
            # r is list of sockets with incoming data 
            # w is list of sockets ready to accept outgoing data 
            # e is list of sockets with an error state 
            r,w,e = select.select(self.read_waiting,self.write_waiting,[],timeout)
            for fd in r: self.schedule(self.read_waiting.pop(fd))
            for fd in w: self.schedule(self.write_waiting.pop(fd))
    # Put I/O polling in its own task 
    def iotask(self):
        while True:
            if self.ready.empty():
                self.iopoll(None)
            else:
                self.iopoll(0)
            yield 
    
    def new(self, target): # Introduces a new task to the scheduler 
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask 
        self.schedule(newtask)
        return newtask.tid 
    
    def schedule(self, task):
        self.ready.put(task) # Put a task onto the ready queue, This makes it available to run 
    
    def exit(self, task): # Remove the task from the scheduler's task map 
        print("Task %d terminated" %task.tid)
        del self.taskmap[task.tid]
        # Notify other tasks waiting for exit 
        # When a task exits, we pop a list of all waiting tasks off out of the waiting area and reschedule them
        for task in self.exit_waiting.pop(task.tid, []):
            self.schedule(task)
    
    def waitforexit(self, task, waittid): # A utility method that makes a task wait for another task. It puts the task in the waiting area 
        if waittid in self.taskmap:
            self.exit_waiting.setdefault(waittid,[]).append(task)
            return True 
        else:
            return False 
        
    def mainloop(self): # The main scheduler loop. It pulls tasks off the queue and runs them to the next yield 
        self.new(self.iotask()) # Launch I/O polls 
        while self.taskmap:
            task = self.ready.get()
            try: # Catch task exit and cleanup
                result = task.run() # Look at the result yielded by the task. If it's a SystemCall, do some setup and run the system call on behalf of the task
                if isinstance(result, SystemCall):
                    result.task = task  # These attributes hold information about the environment (current task and scheduler)
                    result.sched = self
                    result.handle()
                    continue 
            except StopIteration:
                self.exit(task)
                continue
            self.schedule(task)

class SystemCall(object): # System Call base class. All system operations will be operations will be implemented by inheriting from this class 
    def handle(self):
        pass 

# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)

# Create a another system call 
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target 
    
    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid 
        self.sched.schedule(self.task)

# Read/Write Syscalls 
class ReadWait(SystemCall):
    def __init__(self, f):
        self.f = f 
    
    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforread(self.task, fd)

class WriteWait(SystemCall):
    def __init__(self,f):
        self.f = f 
    
    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforwrite(self.task, fd)
        
def handle_client(client, addr):
    print("Connection from", addr)
    while True:
        yield ReadWait(client)
        data = client.recv(65536)
        if not data:
            break 
        yield WriteWait(client)
        client.send(data)
    client.close()
    print("Client closed")
    yield   # Make the function a generator/coroutine 

def server(port):
    print("Server starting")
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(("",port))
    sock.listen(5)
    while True: # The main server loop. Wait for a connection launch a new task to handle each client 
        yield ReadWait(sock)
        client,addr = sock.accept()
        yield NewTask(handle_client(client, addr))

def alive():
    while True:
        print("I'm alive!")
        yield 

sched = Scheduler()
sched.new(alive())
sched.new(server(45001))
sched.mainloop()

I'm alive!
Server starting
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm a

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

...

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

KeyboardInterrupt: 

In [8]:
# Part 8
# The Problem with the Stack 

# A Limitation
# When working with coroutines, you can't write subroutine functions that yield ()
# The yield statement can only be used to suspend a coroutine at the top-most level 

# Coroutine Trampolining 

# A subroutine 
def add(x,y):
    yield x+y 

# A function that calls a subroutine 
def main():
    r = yield add(2,2)
    print(r)
    yield 

# Here is very simpler scheduler code 
def run():
    m  = main()
    # An example of a "trampoline"
    sub = m.send(None)
    result = sub.send(None)
    m.send(result)

run()

4


In [11]:
# An Echo Server Attempt 
import socket 
import types 
from queue import Queue
# This module provides access to the select() and poll() functions 
import select # Waiting for I/O completion 

# A task object 
class Task(object):
    taskid = 0
    def __init__(self, target):
        Task.taskid += 1
        self.tid      = Task.taskid  # Task ID
        self.target   = target       # Target coroutine 
        self.sendval  = None         # Value to send 
        self.stack = []              # If you're going to have subroutines, you first need a "call stack"
    
    def run(self):
        while True:
            try:
                result = self.target.send(self.sendval)
                if isinstance(result, SystemCall): 
                    return result 
                if isinstance(result,types.GeneratorType):
                    self.stack.append(self.target) # Push the current coroutine on the stack, loop back to the top, and call the new coroutine
                    self.sendval = None
                    self.target = result 
                else:
                    if not self.stack: 
                        return 
                    self.sendval = result 
                    self.target = self.stack.pop()
            except StopIteration:
                if not self.stack: 
                    raise 
                self.sendval = None
                self.target = self.stack.pop() 
                    
# The Scheduler 
class Scheduler(object):
    def __init__(self):
        self.ready = Queue() # A queue of tasks that are ready to run 
        self.taskmap = {} # A dictinary that keeps track of all active tasks (each task has a unique integer task ID)
        self.read_waiting = {} # holding areas for tasks blocking on I/O. These are dictionaries mapping file descriptors to tasks
        self.write_waiting = {} 
        self.exit_waiting = {} # This is a holding area for tasks that are waiting. A dict mapping task ID to tasks waiting for exit.
    
    def waitforread(self, task, fd):
        self.read_waiting[fd] = task 
    
    def waitforwrite(self, task, fd):
        self.write_waiting[fd] = task 
    
    def iopoll(self, timeout): # I/O Polling. Use select() to determine which file descriptors can be used. Unblock any associated task
        if self.read_waiting or self.write_waiting:
            # Poll for I/O activity 
            # r is list of sockets with incoming data 
            # w is list of sockets ready to accept outgoing data 
            # e is list of sockets with an error state 
            r,w,e = select.select(self.read_waiting,self.write_waiting,[],timeout)
            for fd in r: self.schedule(self.read_waiting.pop(fd))
            for fd in w: self.schedule(self.write_waiting.pop(fd))
                
    # Put I/O polling in its own task 
    def iotask(self):
        while True:
            if self.ready.empty():
                self.iopoll(None)
            else:
                self.iopoll(0)
            yield 
    
    def new(self, target): # Introduces a new task to the scheduler 
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask 
        self.schedule(newtask)
        return newtask.tid 
    
    def schedule(self, task):
        self.ready.put(task) # Put a task onto the ready queue, This makes it available to run 
    
    def exit(self, task): # Remove the task from the scheduler's task map 
        print("Task %d terminated" %task.tid)
        del self.taskmap[task.tid]
        # Notify other tasks waiting for exit 
        # When a task exits, we pop a list of all waiting tasks off out of the waiting area and reschedule them
        for task in self.exit_waiting.pop(task.tid, []):
            self.schedule(task)
    
    def waitforexit(self, task, waittid): # A utility method that makes a task wait for another task. It puts the task in the waiting area 
        if waittid in self.taskmap:
            self.exit_waiting.setdefault(waittid,[]).append(task)
            return True 
        else:
            return False 
        
    def mainloop(self): # The main scheduler loop. It pulls tasks off the queue and runs them to the next yield 
        self.new(self.iotask()) # Launch I/O polls 
        while self.taskmap:
            task = self.ready.get()
            try: # Catch task exit and cleanup
                result = task.run() # Look at the result yielded by the task. If it's a SystemCall, do some setup and run the system call on behalf of the task
                if isinstance(result, SystemCall):
                    result.task = task  # These attributes hold information about the environment (current task and scheduler)
                    result.sched = self
                    result.handle()
                    continue 
            except StopIteration:
                self.exit(task)
                continue
            self.schedule(task)

class SystemCall(object): # System Call base class. All system operations will be operations will be implemented by inheriting from this class 
    def handle(self):
        pass 

# Return a task's ID number
class GetTid(SystemCall):
    def handle(self):
        self.task.sendval = self.task.tid
        self.sched.schedule(self.task)

# Create a another system call 
class NewTask(SystemCall):
    def __init__(self, target):
        self.target = target 
    
    def handle(self):
        tid = self.sched.new(self.target)
        self.task.sendval = tid 
        self.sched.schedule(self.task)

# Read/Write Syscalls 
class ReadWait(SystemCall):
    def __init__(self, f):
        self.f = f 
    
    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforread(self.task, fd)

class WriteWait(SystemCall):
    def __init__(self,f):
        self.f = f 
    
    def handle(self):
        fd = self.f.fileno()
        self.sched.waitforwrite(self.task, fd)

class Socket(object):
    def __init__(self, sock):
        self.sock = sock 
    
    def accept(self):
        yield ReadWait(self.sock)
        client, addr = self.sock.accept()
        yield Socket(client), addr
    
    def send(self, buffer):
        while buffer:
            yield WriteWait(self.socks)
            len = self.sock.send(buffer)
            buffer = buffer[len:]
    
    def recv(self, maxbytes):
        yield ReadWait(self.sock)
        yield self.sock.recv(maxbytes)
    
    def close(self):
        yield self.sock.close() 
        
def Accept(sock):
    yield ReadWait(sock)
    yield sock.accept()

def Send(sock, buffer):
    while buffer:
        yield WriteWait(sock)
        len = sock.send(buffer)
        buffer = buffer[len:]

def Recv(sock, maxbytes):
    yield ReadWait(sock)
    yield sock.recv(maxbytes)


def handle_client(client, addr):
    print("Connection from", addr)
    while True:
        data = yield client.recv(65536)
        if not data:
            break 
        yield client.send(data)
    print("Client closed")
    yield client.close() 

def server(port):
    print("Server starting")
    rawsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    rawsock.bind(("",port))
    rawsock.listen(5)
    sock = Socket(rawsock)
    while True: # The main server loop. Wait for a connection launch a new task to handle each client 
        client,addr = yield sock.accept()
        yield NewTask(handle_client(client, addr))

def alive():
    while True:
        print("I'm alive!")
        yield 

sched = Scheduler()
sched.new(alive())
sched.new(server(45003))
sched.mainloop()


I'm alive!
Server starting
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm a

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

...

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!
I'm alive!

KeyboardInterrupt: 

In [12]:
# Example: Twisted (echo server)
from twisted.internet.protocol import Protocol, Factory
from twisted.internet import reactor 

class Echo(Protocol):
    def dataReceived(self, data):
        self.transport.write(data)  # An event callback 

def main():
    f = Factory()
    f.protocol = Echo 
    reactor.listenTCP(45005, f)
    reactor.run()

if __name__ == '__main__':
    main()


In [ ]:
# Part 9
# Some Final Words

# Python generators are far more powerful than most people realize 

In [56]:
# https://pyformat.info/
# Using % and .format() for great good!

# Basic formatting
print('%s %s' %('one', 'two'))
print('{} {}'.format('one', 'two'))
print('{1} {0}'.format('one','two'))
print('%d %d' %(1,2))
print('{} {}'.format(1,2))

# Value conversion 
# !s __str__() ; !r __repr__() 

class Data(object):
    def __str__(self):
        return 'str'
    
    def __repr__(self):
        return 'räpr'
    
print('%s %r'%(Data(), Data()))
print('{0!s} {0!r}'.format(Data()))
# Python3 there exists an additional conversion flag that uses the output of repr() uses ascii() 
print('%r %a' %(Data(), Data()))
print('{0!r} {0!a}'.format(Data()))

# Padding and aligning strings 
# Unfortunately the default alignment differs between old and new style formatting. The old style defaults to right aligned while for new style it's left
print('%10s' %('test')) # old style default to right 
print('%-10s' %('test'))
print('{:10}'.format('test')) # new style default to left 
print('{:>10}'.format('test'))
print('{:_<10}'.format('test')) # choose the padding character
print('{:^10}'.format('test')) # center align values
print('{:^6}'.format('zip')) # extra character will be placed on the right side

# Truncating long strings
print('%.5s' %('xylophone')) # . in the format specifies the precision of the output 
print('{:.5}'.format('xylophone'))
print('%-10.5s' %('xylophone')) # Combining truncating and padding 
print('{:10.5}'.format('xylophone'))

# Numbers 
print('%d' %(42,))
print('{:d}'.format(42,))
print('%f' %(3.1415926,))
print('{:f}'.format(3.1415926,))
print('%4d' %(42,))
print('{:4d}'.format(42,))
print('%06.2f' %(3.1415926))
print('{:06.2f}'.format(3.1415926))

# Signed numbers
print('%+d' %(42,))
print('{:+d}'.format(42,))
print('% d' %(-23,)) # space character to indicate that negative numbers should be prefix 
print('{: d}'.format(-23,))
print('% d' %(42,))
print('{:=+5d}'.format(-23,)) # control the position of the sign symbol relative to the padding 

# Named placeholder 
data = {'first': 'Hodor', 'last': 'Hodor!'}
print('%(first)s %(last)s' %(data))
print('{first} {last}'.format(**data))
print('{first} {last}'.format(first='Hodor', last='Hodor!')) # .format() also accept keyword argument 

# Getitem and Getattr 
# New style formatting allows even greater flexibility in accessing nested data structures 
person = {'first': 'Jean-Luc', 'last': 'Picard'}
data = [4, 8, 15, 16, 23, 42]
print('{p[first]} {p[last]}'.format(p=person))
print('{d[4]} {d[5]}'.format(d=data))
# as well as accessing attributes on objects via getattr()
class Plant(object):
    type = 'tree'
    kinds = [{'name':'oak'},{'name':'maple'}]
print('{p.type}'.format(p=Plant()))
print('{p.type}: {p.kinds[0][name]}'.format(p=Plant()))

# Datetime 
from datetime import datetime 
print('{:%Y-%m-%d %H:%M}'.format(datetime(2018, 3, 1, 11, 40)))
# parameterized alignment and width 
print('{:{align}{width}}'.format('test', align='^', width='10'))
# parametrized precision 
print('%.*s = %.*f' %(3, 'Gibberish', 3, 2.7182))
print('{:.{prec}} = {:.{prec}f}'.format('Gibberish', 2.7182, prec=3))
# Width and precision 
print('%*.*f' %(5, 2, 2.7182))
print('{:{width}.{prec}f}'.format(2.7182, width=5, prec=2))
print('{:{prec}} = {:{prec}}'.format('Gibberish', 2.7182, prec='.3'))
dt = datetime(2018,3,1)
print('{:{dfmt} {tfmt}}'.format(dt, dfmt='%Y-%m-%d', tfmt='%H:%M'))

# Custom objects
class HAL9000(object):
    def __format__(self, format):
        if (format == 'open-the-pod-bay-doors'):
            return "I'm afarid I can't do that"
        return 'HAL 9000'

print('{:open-the-pod-bay-doors}'.format(HAL9000()))

one two
one two
two one
1 2
1 2
str räpr
str räpr
räpr r\xe4pr
räpr r\xe4pr
      test
test      
test      
      test
test______
   test   
 zip  
xylop
xylop
xylop     
xylop     
42
42
3.141593
3.141593
  42
  42
003.14
003.14
+42
+42
-23
-23
 42
-  23
Hodor Hodor!
Hodor Hodor!
Hodor Hodor!
Jean-Luc Picard
23 42
tree
tree: oak
2018-03-01 11:40
   test   
Gib = 2.718
Gib = 2.718
 2.72
 2.72
Gib = 2.72
2018-03-01 00:00
I'm afarid I can't do that
